# KY Top100 Daily 수집
* 노래방 인기곡

## 필요 라이브러리 로드

In [40]:
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime
import os
import requests
from numpy import nan

## 필요한 파일 확인 및 설치

### 현재 경로 확인

In [2]:
# 현재 경로 확인
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/data/Wantreez/Crawling/music'

In [3]:
# 수집한 파일 저장할 폴더 생성
crawled_folder_path = code_path + '/crawled_data/live_ky'
os.makedirs(crawled_folder_path, exist_ok=True)

## 페이지 접속

In [58]:
url1 = 'https://kysing.kr/popular/?range=1'
url2 = 'https://kysing.kr/popular/?range=2'

In [59]:
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
data = requests.get(url1, headers=headers)

In [60]:
# 수프에 담기
soup = bs(data.text, 'lxml')
soup.title.text

'인기곡 ⋆ KYSing'

## 순위 날짜 수집

In [61]:
# 순위 날짜
rank_date = datetime.today().strftime('%Y-%m-%d')
rank_date

'2022-05-02'

In [62]:
repeat_rank_date_list = []
for i in range(100):
    repeat_rank_date_list.append(rank_date)
len(repeat_rank_date_list)

100

In [63]:
ul_soup = soup.find_all('ul', 'popular_chart_list clear')[1:]
len(ul_soup)

50

## 곡 제목 수집

In [64]:
# song_title
song_titles = ul_soup[0].find('span', 'tit').text
song_titles

'사랑은 늘 도망가 (드라마 신사와 아가씨)'

In [65]:
title_list = []
for one in ul_soup:
    temp = one.find('span', 'tit').text.strip()
    title_list.append(temp)
len(title_list)

50

In [66]:
title_list[45:]

['Heaven', '너를 위해(영화"동감")', '서쪽 하늘', '남자를 몰라', '라라라']

## 가수 수집

In [67]:
# artist
artist = ul_soup[0].find('span', 'tit mo-art').text
artist

'임영웅'

In [68]:
artist_list = []
for one in ul_soup:
    temp = one.find('span', 'tit mo-art').text.strip()
    artist_list.append(temp)
len(artist_list)

50

In [69]:
artist_list[40:]

['하동균',
 '김현정',
 '김건모',
 'M.C THE MAX !',
 '정동하',
 '김현성',
 '임재범',
 '울랄라세션',
 '버즈',
 'SG워너비']

## 앨범 수집

In [70]:
album_list = []
for i in range(100):
    album_list.append(nan)
len(album_list)

100

In [71]:
album_list[:5]

[nan, nan, nan, nan, nan]

## 순위 생성

In [72]:
# rank
rank_list = []
for i in range(1, 101):
    rank_list.append(i)

len(rank_list)

100

### 2번째 페이지(51~100위 수집)

In [73]:
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}
data = requests.get(url2, headers=headers)

In [74]:
# 수프에 담기
soup = bs(data.text, 'lxml')
soup.title.text

'인기곡 ⋆ KYSing'

In [75]:
ul_soup = soup.find_all('ul', 'popular_chart_list clear')[1:]
len(ul_soup)

50

## 곡 제목 수집

In [76]:
# song_title
song_titles = ul_soup[0].find('span', 'tit').text
song_titles

'그대라는 사치'

In [77]:
for one in ul_soup:
    temp = one.find('span', 'tit').text.strip()
    title_list.append(temp)
len(title_list)

100

In [78]:
title_list[95:]

['까만안경', '조금 취했어 (Prod. 2Soo)', '사계 (하루살이)', '붉은 노을', '그날처럼']

## 가수 수집

In [79]:
# artist
artist = ul_soup[0].find('span', 'tit mo-art').text
artist

'한동근            '

In [80]:
for one in ul_soup:
    temp = one.find('span', 'tit mo-art').text.strip()
    artist_list.append(temp)
len(artist_list)

100

In [81]:
artist_list[95:]

['이루', '임재현', 'M.C THE MAX!', '빅뱅', '장덕철']

## df 생성

In [82]:
# make df
dict = {'날짜':repeat_rank_date_list, '순위':rank_list, '곡':title_list, '가수': artist_list}
df = pd.DataFrame(dict)

In [83]:
df

,날짜,순위,곡,가수
0,2022-05-02,1,사랑은 늘 도망가 (드라마 신사와 아가씨),임영웅
1,2022-05-02,2,소주 한잔,임창정
2,2022-05-02,3,취중 고백,김민석(멜로망스)
3,2022-05-02,4,Tears,소찬휘
4,2022-05-02,5,가시,버즈
...,...,...,...,...
95,2022-05-02,96,까만안경,이루
96,2022-05-02,97,조금 취했어 (Prod. 2Soo),임재현
97,2022-05-02,98,사계 (하루살이),M.C THE MAX!
98,2022-05-02,99,붉은 노을,빅뱅


## 파일 생성

In [85]:
# make excel
today_date = datetime.today().strftime("%Y%m%d_%H%M%S")
file_name = f'live_ky_{today_date}.xlsx'
path = crawled_folder_path + '/'
df.to_excel(path + file_name, index=False, encoding='utf-8')

print(f"{file_name} 파일 생성 완료")

live_ky_20220502_154757.xlsx 파일 생성 완료


In [101]:
df.to_excel(f'\\\\192.168.100.178/d/용역/2022/2022년 음원사재기 관련 모니터링 위탁용역/음원순위_크롤링/live_ky/{file_name}', index=False, encoding='utf-8')

# 코드 마지막